In [11]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import AutoProcessor, AutoModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/siglip-base-patch16-224"

processor = AutoProcessor.from_pretrained(model_name)

model = AutoModel.from_pretrained(model_name).to(device).eval()


In [13]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"

canon_df = pd.read_csv(file_path)

canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

cols = ['ImageFilePath', 'Genus']

df = canon_df[cols]


In [14]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        features = model.get_image_features(pixel_values=inputs["pixel_values"])
    
    return features.cpu().numpy()
    

In [15]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"])])

le = LabelEncoder()

y = le.fit_transform(df["Genus"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


100%|██████████| 1622/1622 [05:06<00:00,  5.30it/s]


In [16]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', max_iter=200, random_state=seed)
}


predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

NaiveBayes                | Acc: 95.08% | Prec: 95.29% | Rec: 95.08% | F1: 95.11% | Bal Acc: 96.80% | MCC: 0.9003
LogisticRegression        | Acc: 98.77% | Prec: 98.77% | Rec: 98.77% | F1: 98.77% | Bal Acc: 96.58% | MCC: 0.9743
SVMLinear                 | Acc: 98.77% | Prec: 98.77% | Rec: 98.77% | F1: 98.77% | Bal Acc: 96.58% | MCC: 0.9743
SVMPolynomial             | Acc: 72.92% | Prec: 79.67% | Rec: 72.92% | F1: 66.98% | Bal Acc: 51.80% | MCC: 0.4067
SVMRadialBasis            | Acc: 98.46% | Prec: 98.48% | Rec: 98.46% | F1: 98.40% | Bal Acc: 91.18% | MCC: 0.9677
NearestNeighbor           | Acc: 97.54% | Prec: 97.57% | Rec: 97.54% | F1: 97.54% | Bal Acc: 95.78% | MCC: 0.9488
RandomForest              | Acc: 93.54% | Prec: 93.77% | Rec: 93.54% | F1: 92.18% | Bal Acc: 66.85% | MCC: 0.8620
MLP_Baseline              | Acc: 98.77% | Prec: 98.82% | Rec: 98.77% | F1: 98.78% | Bal Acc: 99.37% | MCC: 0.9748


In [17]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax


In [18]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.950769,0.952873,0.950769,0.951146,0.967971,0.900317
LogisticRegression,LogisticRegression,0.987692,0.987692,0.987692,0.987692,0.965795,0.974286
SVMLinear,SVMLinear,0.987692,0.987692,0.987692,0.987692,0.965795,0.974286
SVMPolynomial,SVMPolynomial,0.729231,0.796744,0.729231,0.669806,0.518028,0.406731
SVMRadialBasis,SVMRadialBasis,0.984615,0.984788,0.984615,0.98401,0.911819,0.967719
NearestNeighbor,NearestNeighbor,0.975385,0.975736,0.975385,0.975415,0.957787,0.948836
RandomForest,RandomForest,0.935385,0.937672,0.935385,0.921808,0.668517,0.862034
MLP_Baseline,MLP_Baseline,0.987692,0.988193,0.987692,0.987762,0.993681,0.974848


In [22]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/30.SigLIP-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/30.SigLIP-linear-probing-genus-metrics.csv", index=False)